<a href="https://colab.research.google.com/github/PaoNguyen191/AI-Powered_Marketing_Brochures/blob/main/Brochure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get("open_ai_key")

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import display, Markdown
from typing import List
from openai import OpenAI
import google.generativeai

In [ ]:
import gradio as gr

In [ ]:
openai = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
system_message = "You are a helpful assistant"

In [ ]:
def message_gpt(prompt):
  messages = [
      {"role": "system", "content": system_message},
      {"role": "user", "content": prompt}
  ]
  completion=openai.chat.completions.create(
      model='gpt-4o',
      messages=messages
  )
  return completion.choices[0].message.content

In [ ]:
message_gpt("What is today's date?")

"I'm sorry, I can't provide today's date as my training only goes up until October 2023 and I don't have real-time capabilities. However, you can check today's date using your computer, smartphone, or any other calendar."

In [ ]:
def shout(text):
  print(f"Shout has been called with input {text}")
  return text.upper()

In [ ]:
shout("hello")

Shout has been called with input hello


'HELLO'

In [13]:
view = gr.Interface(
      fn=message_gpt,
      inputs=[gr.Textbox(label="Your message:")],
      outputs=[gr.Markdown(label="Response:")],
      allow_flagging="never"
    )
view.launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://67c305a4858100cf23.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [14]:
def stream_gpt(prompt):
  messages = [
      {"role": "system", "content": system_message},
      {"role": "user", "content": prompt}
  ]
  stream = openai.chat.completions.create(
      model='gpt-4o',
      messages=messages,
      stream=True
  )
  result=""
  for chunk in stream:
    result+=chunk.choices[0].delta.content or ""
    yield result

In [17]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    allow_flagging="never"
)
view.launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://354a7b0cb8698b8499.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Building a company brochure generator**

In [19]:
class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str
    link: List[str]
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        self.boday = response.content
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [27]:
system_prompt="You are an assistant that analyzes the contents of a company website landing page \
and create a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [34]:
def stream_brochure(company_name, url):
  prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
  prompt += Website(url).get_contents()
  stream = openai.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "system", "content": "You are a brochure writer."},
              {"role": "user", "content": prompt}],
    stream=True
  )

  result=""
  for chunk in stream:
    result+=chunk.choices[0].delta.content or ""
    yield result


In [35]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[gr.Textbox(label="Company Name:"),
            gr.Textbox(label="URL:")],
    outputs=[gr.Markdown(label="Brochure:")],
    allow_flagging="never"
)
view.launch()

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9c6a0dcec959b13f36.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
